In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
%pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,270 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,347 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,005 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadco

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales= spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, ignoreLeadingWhiteSpace=True)


In [4]:
# 2. Create a temporary view of the DataFrame.
home_sales.createOrReplaceTempView("sales_data")


In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
avg_price_4bed = spark.sql("""
    SELECT ROUND(AVG(price), 2) AS AveragePrice, YEAR(date) AS Year
    FROM sales_data
    WHERE bedrooms = 4
    GROUP BY YEAR(date)
    ORDER BY Year DESC
""")

#Show the results
avg_price_4bed.show()


+------------+----+
|AveragePrice|Year|
+------------+----+
|   296363.88|2022|
|   301819.44|2021|
|   298353.78|2020|
|    300263.7|2019|
+------------+----+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
avg_price_3bed3bath = spark.sql("""
    SELECT YEAR(date_built) AS YearBuilt, ROUND(AVG(price), 2) AS AveragePrice
    FROM sales_data
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY YEAR(date_built)
    ORDER BY YearBuilt DESC
""")

# Show the results
avg_price_3bed3bath.show()


+---------+------------+
|YearBuilt|AveragePrice|
+---------+------------+
|     2017|   292676.79|
|     2016|   290555.07|
|     2015|    288770.3|
|     2014|   290852.27|
|     2013|   295962.27|
|     2012|   293683.19|
|     2011|   291117.47|
|     2010|   292859.62|
+---------+------------+



In [7]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
avg_price_3b3b2f = spark.sql("""
    SELECT YEAR(date_built) AS YearBuilt, ROUND(AVG(price), 2) AS AveragePrice
    FROM sales_data
    WHERE bedrooms = 3
    AND bathrooms = 3
    AND floors = 2
    AND sqft_living >= 2000
    GROUP BY YEAR(date_built)
    ORDER BY YearBuilt DESC
""")

# Show the result
avg_price_3b3b2f.show()


+---------+------------+
|YearBuilt|AveragePrice|
+---------+------------+
|     2017|   280317.58|
|     2016|    293965.1|
|     2015|   297609.97|
|     2014|   298264.72|
|     2013|   303676.79|
|     2012|   307539.97|
|     2011|   276553.81|
|     2010|   285010.22|
+---------+------------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
    SELECT view, ROUND(AVG(price),2) AS AveragePrice
    FROM sales_data
    GROUP BY view
    HAVING ROUND(AVG(price),2) >= 350000
    ORDER BY view desc
""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+------------+
|view|AveragePrice|
+----+------------+
|  99|  1061201.42|
|  98|  1053739.33|
|  97|  1129040.15|
|  96|  1017815.92|
|  95|   1054325.6|
|  94|   1033536.2|
|  93|  1026006.06|
|  92|   970402.55|
|  91|  1137372.73|
|  90|  1062654.16|
|  89|  1107839.15|
|  88|  1031719.35|
|  87|   1072285.2|
|  86|  1070444.25|
|  85|  1056336.74|
|  84|  1117233.13|
|  83|  1033965.93|
|  82|   1063498.0|
|  81|  1053472.79|
|  80|   991767.38|
+----+------------+
only showing top 20 rows

--- 1.2750928401947021 seconds ---


In [10]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("sales_data")


In [12]:
# 8. Check if the table is cached.
spark.catalog.isCached('sales_data')

True

In [13]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
SELECT view, ROUND(AVG(price), 2) AS AveragePrice
FROM sales_data
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view desc
"""
).show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+------------+
|view|AveragePrice|
+----+------------+
|  99|  1061201.42|
|  98|  1053739.33|
|  97|  1129040.15|
|  96|  1017815.92|
|  95|   1054325.6|
|  94|   1033536.2|
|  93|  1026006.06|
|  92|   970402.55|
|  91|  1137372.73|
|  90|  1062654.16|
|  89|  1107839.15|
|  88|  1031719.35|
|  87|   1072285.2|
|  86|  1070444.25|
|  85|  1056336.74|
|  84|  1117233.13|
|  83|  1033965.93|
|  82|   1063498.0|
|  81|  1053472.79|
|  80|   991767.38|
+----+------------+
only showing top 20 rows

--- 2.494096279144287 seconds ---


In [14]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales.write.partitionBy('date_built').parquet('parquet_home_sales',mode='overwrite')

In [15]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet('parquet_home_sales')

In [16]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("parquet_sales_data")

In [17]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

parquet_result = spark.sql("""
SELECT view, ROUND(AVG(price), 2) AS AveragePrice
FROM sales_data
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view desc
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+------------+
|view|AveragePrice|
+----+------------+
|  99|  1061201.42|
|  98|  1053739.33|
|  97|  1129040.15|
|  96|  1017815.92|
|  95|   1054325.6|
|  94|   1033536.2|
|  93|  1026006.06|
|  92|   970402.55|
|  91|  1137372.73|
|  90|  1062654.16|
|  89|  1107839.15|
|  88|  1031719.35|
|  87|   1072285.2|
|  86|  1070444.25|
|  85|  1056336.74|
|  84|  1117233.13|
|  83|  1033965.93|
|  82|   1063498.0|
|  81|  1053472.79|
|  80|   991767.38|
+----+------------+
only showing top 20 rows

--- 0.7082946300506592 seconds ---


In [18]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("sales_data")

In [19]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached("sales_data"):
    print("sales_data is still cached.")
else:
    print("sales_data is no longer cached.")


sales_data is no longer cached.
